In [3]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from tensorflow.keras import layers, models


In [4]:
# Paths for your dataset folders
cat_folder = '/content/drive/MyDrive/data/Cat'
dog_folder = '/content/drive/MyDrive/data/Dog'


In [5]:
def remove_corrupted_images(folder):
    num_deleted = 0
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            img = Image.open(file_path)  # Try to open the image
            img.verify()  # Verify that it is a valid image
        except (IOError, SyntaxError):
            print(f"Deleting corrupted image: {file_path}")
            os.remove(file_path)
            num_deleted += 1
    print(f"{num_deleted} corrupted images removed from {folder}.")

# Clean both cat and dog image folders
remove_corrupted_images(cat_folder)
remove_corrupted_images(dog_folder)

Deleting corrupted image: /content/drive/MyDrive/data/Cat/Thumbs.db
Deleting corrupted image: /content/drive/MyDrive/data/Cat/666.jpg
2 corrupted images removed from /content/drive/MyDrive/data/Cat.
Deleting corrupted image: /content/drive/MyDrive/data/Dog/Thumbs.db


/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Deleting corrupted image: /content/drive/MyDrive/data/Dog/11702.jpg
2 corrupted images removed from /content/drive/MyDrive/data/Dog.


In [7]:
# ImageDataGenerator for data augmentation and scaling
data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [9]:
# Train and validation sets
train_data = data_gen.flow_from_directory(
    '/content/drive/MyDrive/data',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

Found 20000 images belonging to 2 classes.


In [11]:
val_data = data_gen.flow_from_directory(
    '/content/drive/MyDrive/data',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


Found 4998 images belonging to 2 classes.


In [13]:
# Build the CNN model
model = models.Sequential()

# Convolutional layers
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten the layers and add dense layers
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))


In [14]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [15]:
# Train the model
history = model.fit(
    train_data,
    steps_per_epoch=train_data.samples // train_data.batch_size,
    epochs=15,
    validation_data=val_data,
    validation_steps=val_data.samples // val_data.batch_size
)


Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 92/625 ━━━━━━━━━━━━━━━━━━━━ 16:23 2s/step - accuracy: 0.4870 - loss: 0.7079

/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))


625/625 ━━━━━━━━━━━━━━━━━━━━ 1472s 2s/step - accuracy: 0.5316 - loss: 0.6906 - val_accuracy: 0.5244 - val_loss: 0.6766
Epoch 2/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 162us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8333 - val_loss: 0.4654
Epoch 3/15


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


625/625 ━━━━━━━━━━━━━━━━━━━━ 1221s 2s/step - accuracy: 0.6330 - loss: 0.6421 - val_accuracy: 0.6815 - val_loss: 0.5875
Epoch 4/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 235us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.4174
Epoch 5/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 1186s 2s/step - accuracy: 0.6879 - loss: 0.5849 - val_accuracy: 0.7256 - val_loss: 0.5363
Epoch 6/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 138us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8333 - val_loss: 0.3451
Epoch 7/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 1212s 2s/step - accuracy: 0.7290 - loss: 0.5375 - val_accuracy: 0.7424 - val_loss: 0.5277
Epoch 8/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 131us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.2651
Epoch 9/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 1156s 2s/step - accuracy: 0.7543 - loss: 0.5005 - val_accuracy: 0.7903 - val_loss: 0.4512
Epoch 10/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 129us/step - accuracy: 0.0000e+00

In [17]:
#  Save the trained model
model.save('cat_dog_classifier.h5')